In [ ]:
import torch
import torch.nn as nn
import math

In [ ]:
class InputEmbeddings(nn.Module):

    def __init__(self, d_model: int, vocab_size: int) -> None:
        super().__init__()
        self.d_model = d_model
        self.vocab_size = vocab_size
        self.embedding = nn.Embedding(vocab_size, d_model)

    def forward(self, x):
        # (batch, seq_len) --> (batch, seq_len, d_model)
        # Multiply by sqrt(d_model) to scale the embeddings according to the paper
        return self.embedding(x) * math.sqrt(self.d_model)

In [ ]:
class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, seq_len: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model
        self.seq_len = seq_len
        self.dropout = nn.Dropout(dropout)
        # Create a matrix of shape (seq_len, d_model)
        pe = torch.zeros(seq_len, d_model)
        # Create a vector of shape (seq_len)
        position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1) # (seq_len, 1)
        # Create a vector of shape (d_model)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # (d_model / 2)
        # Apply sine to even indices
        pe[:, 0::2] = torch.sin(position * div_term) # sin(position * (10000 ** (2i / d_model))
        # Apply cosine to odd indices
        pe[:, 1::2] = torch.cos(position * div_term) # cos(position * (10000 ** (2i / d_model))
        # Add a batch dimension to the positional encoding
        pe = pe.unsqueeze(0) # (1, seq_len, d_model)
        # Register the positional encoding as a buffer
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + (self.pe[:, :x.shape[1], :]).requires_grad_(False) # (batch, seq_len, d_model)
        return self.dropout(x)

In [ ]:
import torch
import torch.nn as nn

class LayerNormalization(nn.Module):

    def __init__(self, features: int, eps: float = 1e-6) -> None:
        super().__init__()
        self.eps = eps
        self.alpha = nn.Parameter(torch.ones(features))  # alpha is a learnable parameter
        self.bias = nn.Parameter(torch.zeros(features))  # bias is a learnable parameter

    def forward(self, x):
        # x: (batch, seq_len, hidden_size)
        mean = x.mean(dim=-1, keepdim=True)  # (batch, seq_len, 1), keep dimension for broadcasting
        std = x.std(dim=-1, keepdim=True)   # (batch, seq_len, 1), keep dimension for broadcasting
        # eps prevents division by zero or very small std values
        return self.alpha * (x - mean) / (std + self.eps) + self.bias


In [ ]:
class FeedForwardBlock(nn.Module):

    def __init__(self, d_model: int, d_ff: int, dropout: float) -> None:
        super().__init__()
        self.linear_1 = nn.Linear(d_model, d_ff) # w1 and b1
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model) # w2 and b2

    def forward(self, x):
        # (batch, seq_len, d_model) --> (batch, seq_len, d_ff) --> (batch, seq_len, d_model)
        return self.linear_2(self.dropout(torch.relu(self.linear_1(x))))

In [ ]:
class MultiHeadAttentionBlock(nn.Module):

    def __init__(self, d_model: int, h: int, dropout: float) -> None:
        super().__init__()
        self.d_model = d_model # Embedding vector size
        self.h = h # Number of heads
        # Make sure d_model is divisible by h
        assert d_model % h == 0, "d_model is not divisible by h"

        self.d_k = d_model // h # Dimension of vector seen by each head
        self.w_q = nn.Linear(d_model, d_model, bias=False) # Wq
        self.w_k = nn.Linear(d_model, d_model, bias=False) # Wk
        self.w_v = nn.Linear(d_model, d_model, bias=False) # Wv
        self.w_o = nn.Linear(d_model, d_model, bias=False) # Wo
        self.dropout = nn.Dropout(dropout)

    @staticmethod
    def attention(query, key, value, mask, dropout: nn.Dropout):
        d_k = query.shape[-1]
        # Just apply the formula from the paper
        # (batch, h, seq_len, d_k) --> (batch, h, seq_len, seq_len)
        attention_scores = (query @ key.transpose(-2, -1)) / math.sqrt(d_k)
        if mask is not None:
            # Write a very low value (indicating -inf) to the positions where mask == 0
            attention_scores.masked_fill_(mask == 0, -1e9)
        attention_scores = attention_scores.softmax(dim=-1) # (batch, h, seq_len, seq_len) # Apply softmax
        if dropout is not None:
            attention_scores = dropout(attention_scores)
        # (batch, h, seq_len, seq_len) --> (batch, h, seq_len, d_k)
        # return attention scores which can be used for visualization
        return (attention_scores @ value), attention_scores

    def forward(self, q, k, v, mask):
        query = self.w_q(q) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        key = self.w_k(k) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        value = self.w_v(v) # (batch, seq_len, d_model) --> (batch, seq_len, d_model)

        # (batch, seq_len, d_model) --> (batch, seq_len, h, d_k) --> (batch, h, seq_len, d_k)
        query = query.view(query.shape[0], query.shape[1], self.h, self.d_k).transpose(1, 2)
        key = key.view(key.shape[0], key.shape[1], self.h, self.d_k).transpose(1, 2)
        value = value.view(value.shape[0], value.shape[1], self.h, self.d_k).transpose(1, 2)

        # Calculate attention
        x, self.attention_scores = MultiHeadAttentionBlock.attention(query, key, value, mask, self.dropout)

        # Combine all the heads together
        # (batch, h, seq_len, d_k) --> (batch, seq_len, h, d_k) --> (batch, seq_len, d_model)
        x = x.transpose(1, 2).contiguous().view(x.shape[0], -1, self.h * self.d_k)

        # Multiply by Wo
        # (batch, seq_len, d_model) --> (batch, seq_len, d_model)
        return self.w_o(x)

In [ ]:
import torch

In [ ]:
# Example input sequences (batch_size=2, seq_len=5)
input_sequences = [
    [1, 2, 3, 0, 0],  # Sequence 1, padded with 0s
    [4, 5, 6, 7, 8]   # Sequence 2, no padding
]
# Convert to tensor
input_tensor = torch.tensor(input_sequences)
# Create a padding mask where 1 indicates a valid position and 0 indicates padding
padding_mask = (input_tensor != 0).unsqueeze(1).unsqueeze(2)
print("Input Tensor:")
print(input_tensor)
print("Padding Mask:")
print(padding_mask)

Input Tensor:
tensor([[1, 2, 3, 0, 0],
        [4, 5, 6, 7, 8]])
Padding Mask:
tensor([[[[ True,  True,  True, False, False]]],


        [[[ True,  True,  True,  True,  True]]]])


In [ ]:
class ResidualConnection(nn.Module):
    def __init__(self, dropout: float) -> None:
        super().__init__()
        self.dropout = nn.Dropout(dropout)
        self.norm = LayerNormalization()

    def forward(self, x, sublayer):
        return x + self.dropout(sublayer(self.norm(x)))

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, self_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout: float) -> None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(2)])

    def forward(self, x, src_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x, x, x, src_mask))
        x = self.residual_connections[1](x, self.feed_forward_block)
        return x

In [ ]:
class Encoder(nn.Module):
    def __init__(self, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization()

    def forward(self, x, mask):
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, self_attention_block: MultiHeadAttentionBlock, cross_attention_block: MultiHeadAttentionBlock, feed_forward_block: FeedForwardBlock, dropout:float)->None:
        super().__init__()
        self.self_attention_block = self_attention_block
        self.cross_attention_block = cross_attention_block
        self.feed_forward_block = feed_forward_block
        self.residual_connections = nn.ModuleList([ResidualConnection(dropout) for _ in range(3)])

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        x = self.residual_connections[0](x, lambda x: self.self_attention_block(x,x,x,tgt_mask))
        x = self.residual_connections[1](x, lambda x: self.cross_attention_block(x, encoder_output, encoder_output, src_mask))
        x = self.residual_connections[2](x, self.feed_forward_block)
        return x

In [ ]:
class Decoder(nn.Module):

    def __init__(self, features: int, layers: nn.ModuleList) -> None:
        super().__init__()
        self.layers = layers
        self.norm = LayerNormalization(features)

    def forward(self, x, encoder_output, src_mask, tgt_mask):
        for layer in self.layers:
            x = layer(x, encoder_output, src_mask, tgt_mask)
        return self.norm(x)

In [ ]:
class ProjectionLayer(nn.Module):

    def __init__(self, d_model, vocab_size) -> None:
        super().__init__()
        self.proj = nn.Linear(d_model, vocab_size)

    def forward(self, x) -> None:
        # (batch, seq_len, d_model) --> (batch, seq_len, vocab_size)
        return self.proj(x)

In [ ]:
class Transformer(nn.Module):

    def __init__(self, encoder: Encoder, decoder: Decoder, src_embed: InputEmbeddings, tgt_embed: InputEmbeddings, src_pos: PositionalEncoding, tgt_pos: PositionalEncoding, projection_layer: ProjectionLayer) -> None:
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.src_pos = src_pos
        self.tgt_pos = tgt_pos
        self.projection_layer = projection_layer

    def encode(self, src, src_mask):
        # (batch, seq_len, d_model)
        src = self.src_embed(src)
        src = self.src_pos(src)
        return self.encoder(src, src_mask)

    def decode(self, encoder_output: torch.Tensor, src_mask: torch.Tensor, tgt: torch.Tensor, tgt_mask: torch.Tensor):
        # (batch, seq_len, d_model)
        tgt = self.tgt_embed(tgt)
        tgt = self.tgt_pos(tgt)
        return self.decoder(tgt, encoder_output, src_mask, tgt_mask)

    def project(self, x):
        # (batch, seq_len, vocab_size)
        return self.projection_layer(x)

In [ ]:
def build_transformer(src_vocab_size: int, tgt_vocab_size: int, src_seq_len: int, tgt_seq_len: int, d_model: int=512, N: int=6, h: int=8, dropout: float=0.1, d_ff: int=2048) -> Transformer:
    # Create the embedding layers
    src_embed = InputEmbeddings(d_model, src_vocab_size)
    tgt_embed = InputEmbeddings(d_model, tgt_vocab_size)

    # Create the positional encoding layers
    src_pos = PositionalEncoding(d_model, src_seq_len, dropout)
    tgt_pos = PositionalEncoding(d_model, tgt_seq_len, dropout)

    # Create the encoder blocks
    encoder_blocks = []
    for _ in range(N):
        encoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        encoder_block = EncoderBlock(d_model, encoder_self_attention_block, feed_forward_block, dropout)
        encoder_blocks.append(encoder_block)

    # Create the decoder blocks
    decoder_blocks = []
    for _ in range(N):
        decoder_self_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        decoder_cross_attention_block = MultiHeadAttentionBlock(d_model, h, dropout)
        feed_forward_block = FeedForwardBlock(d_model, d_ff, dropout)
        decoder_block = DecoderBlock(d_model, decoder_self_attention_block, decoder_cross_attention_block, feed_forward_block, dropout)
        decoder_blocks.append(decoder_block)

    # Create the encoder and decoder
    encoder = Encoder(d_model, nn.ModuleList(encoder_blocks))
    decoder = Decoder(d_model, nn.ModuleList(decoder_blocks))

    # Create the projection layer
    projection_layer = ProjectionLayer(d_model, tgt_vocab_size)

    # Create the transformer
    transformer = Transformer(encoder, decoder, src_embed, tgt_embed, src_pos, tgt_pos, projection_layer)

    # Initialize the parameters
    for p in transformer.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

    return transformer

In [ ]:
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, Dataset

# # Define dummy dataset (replace this with actual data)
# class DummyDataset(Dataset):
#     def __init__(self, src_vocab_size, tgt_vocab_size, seq_len, num_samples=1000):
#         self.src_vocab_size = src_vocab_size
#         self.tgt_vocab_size = tgt_vocab_size
#         self.seq_len = seq_len
#         self.num_samples = num_samples

#     def __len__(self):
#         return self.num_samples

#     def __getitem__(self, idx):
#         # Generate random source and target sequences
#         src = torch.randint(1, self.src_vocab_size, (self.seq_len,))
#         tgt = torch.randint(1, self.tgt_vocab_size, (self.seq_len,))
#         return src, tgt

# # Hyperparameters
# src_vocab_size = 10000
# tgt_vocab_size = 10000
# src_seq_len = 20
# tgt_seq_len = 20
# d_model = 512
# N = 6
# h = 8
# dropout = 0.1
# d_ff = 2048
# batch_size = 32
# num_epochs = 10
# learning_rate = 0.001

# # Initialize dataset and dataloader
# dataset = DummyDataset(src_vocab_size, tgt_vocab_size, src_seq_len)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# # Build model
# transformer = build_transformer(src_vocab_size, tgt_vocab_size, src_seq_len, tgt_seq_len, d_model, N, h, dropout, d_ff)
# criterion = nn.CrossEntropyLoss(ignore_index=0)  # Ignore padding
# optimizer = optim.Adam(transformer.parameters(), lr=learning_rate)

# # Training loop
# for epoch in range(num_epochs):
#     transformer.train()
#     total_loss = 0
#     for src, tgt in dataloader:
#         src_mask = (src != 0).unsqueeze(1).unsqueeze(2)  # Padding mask for source
#         tgt_input = tgt[:, :-1]  # Input to the decoder (excluding <end>)
#         tgt_output = tgt[:, 1:]  # Expected output (excluding <start>)
#         tgt_mask = (tgt_input != 0).unsqueeze(1).unsqueeze(2)

#         # Generate target sequence mask
#         seq_len = tgt_input.size(1)
#         tgt_sub_mask = torch.tril(torch.ones((seq_len, seq_len), device=tgt_input.device)).bool()
#         tgt_mask = tgt_mask & tgt_sub_mask

#         # Forward pass
#         encoder_output = transformer.encode(src, src_mask)
#         decoder_output = transformer.decode(encoder_output, src_mask, tgt_input, tgt_mask)
#         predictions = transformer.project(decoder_output)

#         # Compute loss
#         loss = criterion(predictions.view(-1, tgt_vocab_size), tgt_output.view(-1))
#         total_loss += loss.item()

#         # Backpropagation and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#     print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(dataloader):.4f}")

# print("Training complete!")


In [ ]:
import pandas as pd

data=pd.read_csv("samsum-test.csv",skiprows=1,usecols=[1,2],names=["dialogue","summary"])
data.head()

,dialogue,summary
0,"Hannah: Hey, do you have Betty's number?\nAman...",Hannah needs Betty's number but Amanda doesn't...
1,Eric: MACHINE!\r\nRob: That's so gr8!\r\nEric:...,Eric and Rob are going to watch a stand-up on ...
2,"Lenny: Babe, can you help me with something?\r...",Lenny can't decide which trousers to buy. Bob ...
3,"Will: hey babe, what do you want for dinner to...",Emma will be home soon and she will let Will k...
4,"Ollie: Hi , are you in Warsaw\r\nJane: yes, ju...",Jane is in Warsaw. Ollie and Jane has a party....


In [ ]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer

# Load data
data = pd.read_csv("samsum-test.csv", skiprows=1, usecols=[1, 2], names=["dialogue", "summary"])
data = data.dropna()  # Ensure no missing values

# Split into training and validation sets
train_data, val_data = train_test_split(data, test_size=0.1, random_state=42)

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")  # Replace with your choice of tokenizer
max_length = 128  # Maximum sequence length

def tokenize_function(text):
    """Tokenizes a single text and returns input IDs."""
    return tokenizer(
        text,
        padding="max_length",
        truncation=True,
        max_length=max_length,
        return_tensors="pt"
    )

# Dataset class
class SummarizationDataset(Dataset):
    def __init__(self, dialogues, summaries):
        self.dialogues = dialogues.reset_index(drop=True)
        self.summaries = summaries.reset_index(drop=True)

    def __len__(self):
        return len(self.dialogues)

    def __getitem__(self, idx):
        dialogue = self.dialogues[idx]
        summary = self.summaries[idx]

        src = tokenize_function(dialogue)["input_ids"].squeeze(0)
        tgt = tokenize_function(summary)["input_ids"].squeeze(0)

        # Prepare for teacher forcing
        tgt_input = tgt[:-1]  # Exclude last token for input
        tgt_output = tgt[1:]  # Exclude first token for output

        return {
            "src": src,
            "tgt_input": tgt_input,
            "tgt_output": tgt_output
        }

# Create datasets
train_dataset = SummarizationDataset(train_data["dialogue"], train_data["summary"])
val_dataset = SummarizationDataset(val_data["dialogue"], val_data["summary"])

# DataLoader creation
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, collate_fn=lambda x: collate_batch(x))
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, collate_fn=lambda x: collate_batch(x))

# Collate function
def collate_batch(batch):
    src_batch = [item["src"] for item in batch]
    tgt_input_batch = [item["tgt_input"] for item in batch]
    tgt_output_batch = [item["tgt_output"] for item in batch]

    src_padded = torch.nn.utils.rnn.pad_sequence(src_batch, batch_first=True, padding_value=tokenizer.pad_token_id)
    tgt_input_padded = torch.nn.utils.rnn.pad_sequence(tgt_input_batch, batch_first=True, padding_value=tokenizer.pad_token_id)
    tgt_output_padded = torch.nn.utils.rnn.pad_sequence(tgt_output_batch, batch_first=True, padding_value=tokenizer.pad_token_id)

    return {
        "src": src_padded,
        "tgt_input": tgt_input_padded,
        "tgt_output": tgt_output_padded
    }


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class SummarizationModel(nn.Module):
    def __init__(self, vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, dropout):
        super(SummarizationModel, self).__init__()
        self.transformer = nn.Transformer(d_model=d_model, nhead=nhead, num_encoder_layers=num_encoder_layers,
                                          num_decoder_layers=num_decoder_layers, dim_feedforward=dim_feedforward,
                                          dropout=dropout, batch_first=True)
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.fc_out = nn.Linear(d_model, vocab_size)
        self.src_pad_idx = 0  # Assuming pad token is 0
        self.tgt_pad_idx = 0  # Assuming pad token is 0

    def forward(self, src, tgt_input, src_mask=None, tgt_mask=None):
        # Ensure src and tgt_input are LongTensors before embedding
        src = src.long()  # Cast to LongTensor
        tgt_input = tgt_input.long()  # Cast to LongTensor

        # Embedding the input and target sequences
        src_emb = self.embedding(src)
        tgt_emb = self.embedding(tgt_input)

        # Pass through the transformer
        output = self.transformer(src_emb, tgt_emb, src_mask=src_mask, tgt_mask=tgt_mask)

        # Generate the output predictions
        return self.fc_out(output)

    def generate(self, src, max_length=128):
        # Ensure src is LongTensor before embedding
        src = src.long()  # Cast to LongTensor
        src_emb = self.embedding(src)

        # Create an empty tensor for the target sequence (used for decoding)
        tgt = torch.ones(src.size(0), 1).to(src.device).long()  # Start token for each batch, cast to LongTensor

        for _ in range(max_length):
            tgt_emb = self.embedding(tgt)
            tgt_mask = self.transformer.generate_square_subsequent_mask(tgt.size(1)).to(src.device)

            # Pass through the transformer
            output = self.transformer(src_emb, tgt_emb, tgt_mask=tgt_mask)

            # Get the last output token probabilities
            logits = self.fc_out(output[:, -1, :])

            # Get the next token with the highest probability
            next_token = torch.argmax(logits, dim=-1).unsqueeze(1)

            # Append the token to the target sequence
            tgt = torch.cat((tgt, next_token), dim=1)

            # Break early if we hit an end token
            if next_token.item() == self.tgt_pad_idx:
                break

        return tgt[:, 1:]  # Return the generated tokens (remove the initial start token)


In [ ]:
import torch.optim as optim
import torch.nn as nn

# Hyperparameters
vocab_size = tokenizer.vocab_size
d_model = 512
nhead = 8
num_encoder_layers = 8
num_decoder_layers = 8
dim_feedforward = 2048
dropout = 0.1
learning_rate = 0.001
num_epochs = 10

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model, Loss, Optimizer
model = SummarizationModel(vocab_size, d_model, nhead, num_encoder_layers, num_decoder_layers, dim_feedforward, dropout)
model.to(device)  # Ensure the model is on the same device as the inputs

criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training Loop
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in train_loader:
        # Move data to the correct device
        src = batch['src'].to(device)
        tgt_input = batch['tgt_input'].to(device)
        tgt_output = batch['tgt_output'].to(device)

        # Reset gradients
        optimizer.zero_grad()

        # Create masks (if needed for the transformer)
        src_mask = None
        tgt_mask = None

        # Forward pass
        output = model(src, tgt_input, src_mask, tgt_mask)

        # Calculate loss
        # Calculate loss
        loss = criterion(output.reshape(-1, vocab_size), tgt_output.reshape(-1))


        # Backward pass
        loss.backward()

        # Optimize
        optimizer.step()

        total_loss += loss.item()

    # Print epoch loss
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss / len(train_loader)}")

print("Training complete!")
torch.save(model.state_dict(), "transformer_model.pth")



Epoch 1/10, Loss: 7.91619873046875
Epoch 2/10, Loss: 6.436997056007385
Epoch 3/10, Loss: 6.318034450213115
Epoch 4/10, Loss: 6.295535922050476
Epoch 5/10, Loss: 6.284390926361084
Epoch 6/10, Loss: 6.282669583956401
Epoch 7/10, Loss: 6.27705963452657
Epoch 8/10, Loss: 6.280729293823242
Epoch 9/10, Loss: 6.272269646326701
Epoch 10/10, Loss: 6.2696806987126665
Training complete!


In [ ]:
!pip install rouge_score
from rouge_score import rouge_scorer

def evaluate_model(model, dataloader, tokenizer):
    model.eval()
    scorer = rouge_scorer.RougeScorer(["rouge1", "rouge2", "rougeL"], use_stemmer=True)
    total_scores = {"rouge1": 0, "rouge2": 0, "rougeL": 0}

    with torch.no_grad():
        for batch in dataloader:
            # Assuming 'src', 'tgt_input', and 'tgt_output' are in the batch
            src = batch['src'].to("cuda")  # Input sequence
            tgt_input = batch['tgt_input'].to("cuda")  # Target input sequence
            tgt_output = batch['tgt_output'].to("cuda")  # Target output sequence
            print(type(model))

            # Generate output from the model (using the generate function)
            output = model.generate(src, max_length=128)  # Generate predictions

            # Decode predictions and references (targets)
            for i in range(len(output)):
                # Decode the prediction and reference (target output)
                prediction = tokenizer.decode(output[i], skip_special_tokens=True)
                reference = tokenizer.decode(tgt_output[i], skip_special_tokens=True)

                # Print predictions and references for debugging
                print(f"Prediction: {prediction}")
                print(f"Reference: {reference}")

                # Compute ROUGE scores
                scores = scorer.score(reference, prediction)
                for key in scores:
                    total_scores[key] += scores[key].fmeasure

    # Average the scores across the entire dataset
    for key in total_scores:
        total_scores[key] /= len(dataloader.dataset)

    return total_scores

# Example usage
# Assuming `val_loader` is your validation dataloader and `model` is the trained model
# Also assuming `tokenizer` is your tokenizer used to process the text
scores = evaluate_model(model, val_loader, tokenizer)
print("Evaluation Scores:", scores)


<class '__main__.SummarizationModel'>


RuntimeError: a Tensor with 64 elements cannot be converted to Scalar

In [ ]:
# Import necessary libraries
from transformers import AutoTokenizer
import torch

# Load the trained model and tokenizer
model_path = "transformer_model.pth"  # Update this if you saved the entire model
model = build_transformer(src_vocab_size=1000, tgt_vocab_size=1000, src_seq_len=50, tgt_seq_len=50)
model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
model.eval()  # Set model to evaluation mode

# Define a function for summarization
def generate_summary(input_text, max_summary_length=100):
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")  # Replace with your tokenizer
    input_tokens = tokenizer.encode(input_text, return_tensors="pt", truncation=True, max_length=50)

    # Generate a prediction (simplified; requires model adaptation for real summarization)
    with torch.no_grad():
        output_tokens = model(input_tokens)

    # Decode the output tokens
    summary = tokenizer.decode(output_tokens[0], skip_special_tokens=True)
    return " ".join(summary.split()[:max_summary_length])  # Limit to max summary length

# Create a text input cell
input_text = input("Enter the text you want summarized: ")

# Generate and display the summary
summary = generate_summary(input_text)
print("\nSummary (max 100 words):")
print(summary)
